In [1]:
import pyodbc

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
db_path = r'C:\Users\GILBERT FG\Desktop\Readings\PSP_database.accdb'

conn = pyodbc.connect(
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    rf'DBQ={db_path};'
)

cursor = conn.cursor()

for table in cursor.tables(tableType='TABLE'):
    print(table.table_name)

Block register
Plot monitoring history
Plot-block relation
PSPs
Slope correction factors
Thinning history
TreeData


In [ ]:
query = r"SELECT * FROM TreeData WHERE AreaType = 'Teak' AND Plantations = 'Tain II' AND [Monitoring year] = '2022'"

df = pd.read_sql(query, conn)

df.head()